# Prepare input and Sanity Check

In [1]:
# for reproducibility
import numpy as np
random_seed=42
np.random.seed(random_seed)

## Preparing data (In Orange framework, it is simple and good)

we first prepare the data set

In [2]:
%load_ext autoreload
%autoreload 2
from prepare_dataset import prepare_adult_dataset,prepare_pima_dataset,prepare_diabetes_risk_dataset,prepare_compas_dataset
from utils import encoder_from_datatable


# data_table,test_data_table = prepare_compas_dataset()
data_table,test_data_table = prepare_adult_dataset()

print(data_table.X.shape)
print("now start sanity-check")
print("the first row of the dataset is (orginal form):\n", data_table[0] )
print("the first row of the dataset is (only X data):\n", data_table.X[0])
print("the first row of the dataset is (only y data):\n", data_table.Y[0])

/Users/lucy/Desktop/workspace/Descriptive-Induction-ML/prepare_dataset.py:152: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '?' in df[col].unique():
/Users/lucy/Desktop/workspace/Descriptive-Induction-ML/prepare_dataset.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col][df[col] == '?'] = df[col].value_counts().index[0]


(21815, 12)
now start sanity-check
the first row of the dataset is (orginal form):
 [51.000, Private, Masters, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0.000, 0.000, 50.000, United-States | >50K]
the first row of the dataset is (only X data):
 [51.  3. 12.  2.  3.  0.  4.  1.  0.  0. 50. 38.]
the first row of the dataset is (only y data):
 1.0


## Prepare the blackbox

In [3]:
%load_ext autoreload
%autoreload 2

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer


categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]

scikit_encoder = make_column_transformer(
                            ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
                            remainder = 'passthrough'
                            )
scikit_encoder.fit(data_table.X)

from sklearn.ensemble import  RandomForestClassifier
from sklearn import svm
# c =RandomForestClassifier(n_estimators=15, n_jobs=5)
# c =RandomForestClassifier(n_estimators=15, n_jobs=5,random_state=random_seed)
# c= svm.SVC(gamma='scale')
from sklearn.neural_network import MLPClassifier
c = MLPClassifier(solver='adam', alpha=1e-5,  hidden_layer_sizes=(100, 100,50,50,10), random_state=random_seed,verbose=True)

# c = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
c.fit(scikit_encoder.transform(data_table.X), data_table.Y)


# black_box = predict_fn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Iteration 1, loss = inf
Iteration 2, loss = inf
Iteration 3, loss = inf
Iteration 4, loss = inf
Iteration 5, loss = inf
Iteration 6, loss = inf
Iteration 7, loss = inf
Iteration 8, loss = inf
Iteration 9, loss = inf
Iteration 10, loss = inf
Iteration 11, loss = 0.34209765
Iteration 12, loss = 0.33666717
Iteration 13, loss = 0.33249846
Iteration 14, loss = 0.33583157
Iteration 15, loss = 0.34121604
Iteration 16, loss = 0.33347475
Iteration 17, loss = 0.33353829
Iteration 18, loss = 0.33278725
Iteration 19, loss = 0.33656214
Iteration 20, loss = 0.33564903
Iteration 21, loss = 0.32861201
Iteration 22, loss = 0.33309575
Iteration 23, loss = 0.32922019
Iteration 24, loss = 0.33399863
Iteration 25, loss = 0.32578678
Iteration 26, loss = 0.32775973
Iteration 27, loss = 0.32386613
Iteration 28, loss = 0.32438017
Iteration 29, loss = 0.32307141
Iteration 30, loss = 0.32352079
Iteration 31, loss = 0.32659585

/Users/lucy/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100, 50, 50, 10),
              learning_rate='constant', learning_rate_init=0.001, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [ ]:
print(data_table.X.shape)
print(test_data_table.X.shape)

In [5]:

from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
import sklearn
# predict_fn = lambda x: aml.predict(H2OFrame(x)).as_data_frame().iloc[:,0].to_numpy()

predict_fn = lambda x: c.predict(scikit_encoder.transform(x))

print('Train acc', accuracy_score(data_table.Y, predict_fn(data_table.X)))
print('Test acc', accuracy_score(test_data_table.Y, predict_fn(test_data_table.X)))

print('acc on training set', sklearn.metrics.accuracy_score(data_table.Y, predict_fn(data_table.X)))
print('f1 on training set', sklearn.metrics.f1_score(data_table.Y, predict_fn(data_table.X)))
print('recall on training set', sklearn.metrics.recall_score(data_table.Y, predict_fn(data_table.X)))
print('precision on training set', sklearn.metrics.precision_score(data_table.Y, predict_fn(data_table.X)))

print('acc on test set', sklearn.metrics.accuracy_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('f1 on test set', sklearn.metrics.f1_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('recall on test set', sklearn.metrics.recall_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('precision on test set', sklearn.metrics.precision_score(test_data_table.Y, predict_fn(test_data_table.X)))


Train acc 0.8973641989456795
Test acc 0.8421738321235809
acc on training set 0.8973641989456795
f1 on training set 0.7720655604194239
recall on training set 0.7166887166887167
precision on training set 0.8367166813768756
acc on test set 0.8421738321235809
f1 on test set 0.6372968349016254
recall on test set 0.5843137254901961
precision on test set 0.7008466603951082


In [6]:
black_box = predict_fn

from utils import label_with_blackbox

predicted_data_table = label_with_blackbox(data_table,black_box)

predicted_test_data_table = label_with_blackbox(test_data_table,black_box)


print(predicted_data_table[:5])
print(predicted_data_table.X.shape[0])
print(predicted_test_data_table.X.shape[0])

print('sanity check, of acc,should be 100', accuracy_score(predicted_data_table.Y, predict_fn(predicted_data_table.X)))

[[51.000, Private, Masters, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0.000, 0.000, 50.000, United-States | >50K],
 [46.000, Private, HS-grad, Married-spouse-absent, Prof-specialty, Not-in-family, Asian-Pac-Islander, Male, 0.000, 0.000, 40.000, Philippines | <=50K],
 [25.000, Private, Bachelors, Never-married, Prof-specialty, Own-child, Black, Female, 0.000, 0.000, 55.000, United-States | <=50K],
 [27.000, Private, Assoc-acdm, Never-married, Other-service, Not-in-family, White, Male, 0.000, 0.000, 40.000, United-States | <=50K],
 [47.000, Local-gov, Masters, Married-civ-spouse, Prof-specialty, Husband, White, Male, 7688.000, 0.000, 45.000, United-States | >50K]]
21815
10746
sanity check, of acc,should be 100 1.0


prepare the dataset and the data encoder

## IDS

In [7]:
%load_ext autoreload
%autoreload 2
# from competition_methods_explanation.passive_methods.ids import explain_tabular
from competition_methods_explanation.passive_methods.brs import explain_tabular

iter_max = 1000
explanations= explain_tabular(predicted_data_table, black_box, target_class_idx=1, random_seed=42)

# from competition_methods_explanation.passive_methods.cn2 import cn2_tabular
# explanations = cn2_tabular(data_table,  black_box, target_class_idx = 1, random_seed=42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
defaultdict(<class 'int'>, {'education': 16, 'age': 7, 'relationship': 6, 'occupation': 14, 'marital-status': 7, 'capital-loss': 12, 'workclass': 8, 'race': 5, 'native-country': 41, 'sex': 2, 'capital-gain': 15, 'hours-per-week': 6})


KeyboardInterrupt: 

In [ ]:


from utils import  rule_to_string_BRS_compat,bds_ruleset_predict
for e in explanations:
    print(rule_to_string_BRS_compat(e,data_table.domain,target_class_idx=1))
    
our_prediction = bds_ruleset_predict(explanations,test_data_table.X,domain=test_data_table.domain)
print(sum(our_prediction) )


import sklearn
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from utils import  rule_to_string
print('number of rules:',len(explanations))
# print("new instances",ADS.synthetic_data_table.X.shape[0])


In [ ]:
data_table.domain.class_var

## SBRL

## Non-active ours

In [7]:
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
iter_max = 1000
non_active_explanations,ADS = explain_tabular(predicted_data_table,black_box, target_class_idx=1, random_seed=42,beta=0,lambda_parameter=0.001)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
using Cache
new cached!



Now print Error Log
no error happens


In [8]:
from utils import  rule_to_string,ruleset_predict

print("number of rules",len(non_active_explanations))
explanations = ADS.output_the_best(0.01)
for e in non_active_explanations:
    print(rule_to_string(e,data_table.domain,target_class_idx=1))
    
our_prediction = ruleset_predict(non_active_explanations,predicted_test_data_table.X)

print(sum(our_prediction) )
print(sum(test_data_table.Y) )
print(test_data_table.X.shape )

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

number of rules 41
IF 17.0 <= age <= 24.0 AND education is Doctorate,HS-grad AND marital-status is Never-married AND native-country is Cambodia THEN class=>50K 
IF occupation is Armed-Forces AND 1.0 <= hours-per-week <= 45.0 AND native-country is Cambodia THEN class=>50K 
IF 17.0 <= age <= 25.0 AND occupation is Armed-Forces AND 1.0 <= hours-per-week <= 45.0 AND native-country is Honduras THEN class=>50K 
IF education is Bachelors,Masters,Prof-school AND marital-status is Married-civ-spouse THEN class=>50K 
IF 17.0 <= age <= 57.150000000000006 AND education is Doctorate AND marital-status is Married-civ-spouse,Never-married AND native-country is Japan THEN class=>50K 
IF workclass is Local-gov,State-gov AND native-country is Columbia THEN class=>50K 
IF 17.0 <= age <= 49.85 AND workclass is Self-emp-not-inc,Without-pay AND native-country is Cambodia THEN class=>50K 
IF 17.0 <= age <= 24.0 AND education is Doctorate AND marital-status is Married-civ-spouse,Never-married AND native-count

In [9]:
lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.0001,0.0005,0.001,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]

beta = 0
for lambda_parameter in  lambda_candidates:
    explanations = ADS.output_the_best(lambda_parameter)
    our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)
    f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
    acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
    rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
    pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
    num = len(explanations)
    num_of_instance = ADS.synthetic_data_table.X.shape[0]

    print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)

parameter:  0 0 metrics 113 0.784147557328016 0.9194118741857435 0.7398871119473189 0.8340402969247084 0
parameter:  0 1e-07 metrics 111 0.784147557328016 0.9194118741857435 0.7398871119473189 0.8340402969247084 0
parameter:  0 1e-06 metrics 111 0.784147557328016 0.9194118741857435 0.7398871119473189 0.8340402969247084 0
parameter:  0 1e-05 metrics 111 0.784147557328016 0.9194118741857435 0.7398871119473189 0.8340402969247084 0
parameter:  0 0.0001 metrics 87 0.7824795842613214 0.9182021217197096 0.7436500470366886 0.8255874673629243 0
parameter:  0 0.0005 metrics 66 0.772252610641472 0.9147589800856133 0.7304797742238947 0.8190928270042194 0
parameter:  0 0.001 metrics 41 0.7118118376841561 0.8962404615670947 0.647695202257761 0.7900172117039587 0
parameter:  0 0.002 metrics 3 0.396855110445526 0.8500837520938024 0.24929444967074318 0.9724770642201835 0
parameter:  0 0.0025 metrics 3 0.396855110445526 0.8500837520938024 0.24929444967074318 0.9724770642201835 0
parameter:  0 0.003 metr

In [10]:

from sklearn.ensemble import  RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
c_passive= SVC(gamma='scale')
# c_passive = MLPClassifier(solver='adam', alpha=1e-5,  hidden_layer_sizes=(100, 100,50,50,10), random_state=random_seed,verbose=False)
# c_passive =RandomForestClassifier(n_estimators=15, n_jobs=5)

# c = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
c_passive.fit(scikit_encoder.transform(predicted_data_table.X), predicted_data_table.Y)

black_prediction = c_passive.predict(scikit_encoder.transform(predicted_test_data_table.X))

print(sum(predicted_test_data_table.Y) )
print(test_data_table.X.shape )
import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))

2126.0
(10746, 12)
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.46812683883622097
Blackbox and our, acc 0.8485948259817606
Blackbox and our,recall 0.3367826904985889
Blackbox and our,precision 0.767416934619507


## Ours

In [11]:
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
iter_max = 1000
explanations,ADS = explain_tabular(data_table,  black_box, target_class_idx=1, random_seed=42,beta=0.00001,lambda_parameter=0.001)
# explanations,ADS = explain_tabular(data_table,  black_box, target_class="1", random_seed=42,beta=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
using Cache
new cached!


generating new instance for a batch :  4
generating new instance for a batch :  8
generating new instance for a batch :  39
generating new instance for a batch :  16
generating new instance for a batch :  2
generating new instance for a batch :  17
generating new instance for a batch :  1
generating new instance for a batch :  4
generating new instance for a batch :  1
generating new instance for a batch :  11
generating new instance for a batch :  13
generating new instance for a batch :  1
generating new instance for a batch :  1
generating new instance for a batch :  8
generating new instance for a batch :  1
generating new instance for a batch :  3
generating new instance for a batch :  10
generating new instance for a batch :  1
generating new instance for a batch :  2
generating new instance for a batch :  17
generating new instance for a batch :  4
generating new instance for a batch :  24
generating new instance for a batch :  2
generating new instance for a batch :  19
generat

In [12]:
from utils import  rule_to_string,ruleset_predict

print("number of rules",len(explanations))
# for e in non_active_explanations:
#     print(rule_to_string(e,data_table.domain,target_class_idx=1))
    
our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)

print(sum(our_prediction) )
print(sum(test_data_table.Y) )
print(test_data_table.X.shape )

import sklearn
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

number of rules 11
1609
2550.0
(10746, 12)
Blackbox and our, f1 score 0.6918340026773763
Blackbox and our, acc 0.8928903778150009
Blackbox and our,recall 0.6077140169332079
Blackbox and our,precision 0.8029832193909261


In [13]:
lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.0001,0.0005,0.001,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]

beta = 0
for lambda_parameter in  lambda_candidates:
    explanations = ADS.output_the_best(lambda_parameter)
    our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)
    f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
    acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
    rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
    pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
    num = len(explanations)
    num_of_instance = ADS.synthetic_data_table.X.shape[0]

    print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)

parameter:  0 0 metrics 71 0.7603550295857988 0.9095477386934674 0.7253057384760113 0.7989637305699482 1710
parameter:  0 1e-07 metrics 70 0.7603550295857988 0.9095477386934674 0.7253057384760113 0.7989637305699482 1710
parameter:  0 1e-06 metrics 70 0.7603550295857988 0.9095477386934674 0.7253057384760113 0.7989637305699482 1710
parameter:  0 1e-05 metrics 70 0.7603550295857988 0.9095477386934674 0.7253057384760113 0.7989637305699482 1710
parameter:  0 0.0001 metrics 70 0.7603550295857988 0.9095477386934674 0.7253057384760113 0.7989637305699482 1710
parameter:  0 0.0005 metrics 11 0.6918340026773763 0.8928903778150009 0.6077140169332079 0.8029832193909261 1710
parameter:  0 0.001 metrics 11 0.6918340026773763 0.8928903778150009 0.6077140169332079 0.8029832193909261 1710
parameter:  0 0.002 metrics 11 0.6918340026773763 0.8928903778150009 0.6077140169332079 0.8029832193909261 1710
parameter:  0 0.0025 metrics 11 0.6918340026773763 0.8928903778150009 0.6077140169332079 0.802983219390926

/Users/lucy/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/lucy/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


parameter:  0 0.1 metrics 0 0.0 0.8021589428624605 0.0 0.0 1710
parameter:  0 0.15 metrics 0 0.0 0.8021589428624605 0.0 0.0 1710
parameter:  0 0.2 metrics 0 0.0 0.8021589428624605 0.0 0.0 1710
parameter:  0 0.5 metrics 0 0.0 0.8021589428624605 0.0 0.0 1710


Now in order to test if the approximation is faithful. We generate a huge dataset, and see the agreement of the rules with the blackbox



In [ ]:
import numpy as np
def sample_uniform_whole_space(domain,batch_num=1000,population_size=10000):
    ''' the true sampling procedure
        1. we first extend the specfic region we wish to sampling
                in order to sample, first extend the rule
                for example, if the rule is 1<x<10 -> yes and the domain is 0<x<100,0<y<100
                then the extended rule is 1<x<10,0<y<100 -> yes
                in other words, the un-specified conditions are made to be specific
        2. then we sample one new synthetic at a time and we do it for `batch_size` times
    '''
    from structure import Condition,Rule
    def uniform_sampling(rule_to_uniform_sample):
        ''' uniform sampling. used as the basic procedure'''
        raw_X_columns = [ s.sample(batch_size=population_size) for s in rule_to_uniform_sample.conditions]
        raw_X = np.column_stack(raw_X_columns)
        return raw_X

    conditions = []

    for attri_col, attribute in enumerate(domain.attributes):
        
        if attribute.is_discrete:
            new_condition = Condition(column=attri_col,values= [ i for i in range(len(attribute.values)) ],type='categorical')
        else:
            new_condition = Condition(column=attri_col,max_value=attribute.max,min_value=attribute.min,type='continuous')
        conditions.append(new_condition)
    conditions = sorted(conditions,key=lambda x:x.column )
    rule_whole_space = Rule(conditions,domain,"1")

    synthetic_rows = []

    for _ in range(batch_num):
        population = uniform_sampling(rule_whole_space)

        synthetic_rows.extend(population)
    
    synthetic_instances = np.row_stack(synthetic_rows)

    return synthetic_instances


huge_dataset_X = sample_uniform_whole_space(data_table.domain)


In [ ]:
# print(len(huge_dataset_X))


predict_fn = lambda x: c.predict(scikit_encoder.transform(x))

blackbox_prediction = predict_fn(huge_dataset_X)

def ruleset_predict(ruleset,X):

    curr_covered_or_not = np.zeros(X.shape[0], dtype=np.bool)
    for r in ruleset:
        curr_covered_or_not |= r.evaluate_data(X)
    return curr_covered_or_not



In [ ]:
import operator
def bds_ruleset_predict(ruleset,X,domain):

    curr_covered_or_not = np.zeros(X.shape[0], dtype=np.bool)
    for r in ruleset:
        
        curr_covered = np.ones(X.shape[0], dtype=bool)
        for condition in r.selectors:
            if condition.type == 'categorical':
                tmp = np.logical_or.reduce(
                [np.equal(X[:,condition.column], domain.attributes[condition.column].values.index(v)  ) for v in condition.values ]
                )
            elif condition.type == 'continuous':
                tmp = np.logical_and(
                operator.lt(X[:,condition.column], condition.max),
                operator.gt(X[:,condition.column], condition.min)
                )
                
            curr_covered &= tmp

        curr_covered_or_not |=  curr_covered
    return curr_covered_or_not


BDS_prediction = bds_ruleset_predict(explanations_BDS,huge_dataset_X,data_table.domain)
from utils import rule_to_string_BDS_compat

print(len(explanations_BDS) )

for e in explanations_BDS:
    print(rule_to_string_BDS_compat(e,data_table.domain,1))

print('Blackbox and Bayesian rule list acc', sklearn.metrics.accuracy_score(blackbox_prediction, BDS_prediction) )
print('Blackbox and Bayesian rule list f1 score', sklearn.metrics.f1_score(blackbox_prediction, BDS_prediction) )
print('Blackbox and Bayesian rule list recall', sklearn.metrics.recall_score(blackbox_prediction, BDS_prediction) )
print('Blackbox and Bayesian rule list precision', sklearn.metrics.precision_score(blackbox_prediction, BDS_prediction) )

BDS_prediction = bds_ruleset_predict(explanations_BDS,X_test,data_table.domain)

print('truth and Bayesian rule list acc', sklearn.metrics.accuracy_score(y_test, BDS_prediction) )
print('truth and Bayesian rule list f1 score', sklearn.metrics.f1_score(y_test, BDS_prediction) )
print('truth and Bayesian rule list recall', sklearn.metrics.recall_score(y_test, BDS_prediction) )
print('truth and Bayesian rule list precision', sklearn.metrics.precision_score(y_test, BDS_prediction) )

# print(blackbox_prediction[:5])
# print(BDS_prediction[:5])
# print(our_prediction[:5])